# 식품의약품안전처_의료기기 품목정보 (식의약헬스종합정보)
---
- 작성자 : 안지은
- 실행일 : 2023-01-10

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(key):
    url = 'https://apis.data.go.kr/1471000/MdeqPrdlstInfoService02/getMdeqPrdlstInfoInq02?serviceKey=%s&pageNo=1&type=json'%(key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(key,page):
    
    url = 'https://apis.data.go.kr/1471000/MdeqPrdlstInfoService02/getMdeqPrdlstInfoInq02?serviceKey=%s&pageNo=%s&numOfRows=100&type=json'%(key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

### API 가져오기
---
- 한 페이지 당  100개 씩 제한
- 총 개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [4]:
medi_device = []
cnt = get_total_cnt(key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts(key,i)
    for item in js:
        medi_device.append(item['item'])


 85%|████████▌ | 1578/1854 [25:41<04:29,  1.02it/s]


KeyboardInterrupt: 

### 데이터 프레임 변환
---
- 컬럼명 : API 목록

In [5]:
medi_device_df = pd.json_normalize(medi_device)
medi_device_df.columns = ['의료기기품목일련번호','치수','특성명','제조방법','사용목적','사용방법',
                        '주의사항','유효기관','시험규격','기관지역구분','업종','허가신고구분','의료기기품목허가번호',
                        '품목명','허가일자','취소취하구분코드', '취소취하일자','상품명','분류번호','모델명','등급',
                        '제조의뢰자','제조자','제조국']
medi_device_df

,품명,수량,사용량,재고량,사용내역,구입처
0,브롬화수소산덱스트로메토판,500,0,0,자사에서는 2006.1.8. 현재 표준품 분양을 받지 않았습니다.,예정 구입처 : 식품의약품의약품평가부 마약신경계의약품팀
1,브롬화수소산덱스트로메토판,500,0,0,1,1
2,브롬화수소산덱스트로메토판,500,100,400,2007-03-08 Lot S-0450207(200kg) 자가품질관리용으로 사용,식품의약품의약품평가부마약신경계의약품팀
3,브롬화수소산덱스트로메토판,500,100,400,2007-03-08 LOT S-0450207(200kg) 자가품질관리용으로 사용,식품의약품안전청평가부마약신경계의약품팀
4,브롬화수소산덱스트로메토판,400,100,300,2007-07-27 LOT S-1060407(100kg) 자가품질관리용으로 사용,식품의약품의약품평가부마약신경계의약품팀
...,...,...,...,...,...,...
13386,덱스트로메토르판브롬화수소산염수화물,10519.29,2272.82,8246.47,"상용표준품(S-0220222) : 2,272.82 mg",국전약품 상용표준품
13387,부프레노르핀,200,0,0,Buprenorphine impurity A \n취급승인양 : 200 mg\n수불한...,구입내역 없음
13388,부프레노르핀,15000,0,0,"취급승인양 : 15,000 mg\n수불한양 : -\n실제사용량 : -\n재고량 : -",구입내역 없음
13389,부프레노르핀 포 시스템 슈터빌리티,80,0,70,취급승인양 : 80mg\n수불한양 : 70mg\n실제사용량 : 0 mg\n재고량 :...,EP
